In [1]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [5]:
!pip install ipykernel==4.10
!pip install tokenizers==0.9.4


  Using cached https://files.pythonhosted.org/packages/22/5e/69b0ae84e0cd7c8d4467e0a7b390c560d8c7642493329b15461b41e20fcc/ipykernel-4.10.0-py3-none-any.whl
ERROR: pydeck 0.5.0 has requirement ipykernel>=5.1.2; python_version >= "3.4", but you'll have ipykernel 4.10.0 which is incompatible.
  Found existing installation: ipykernel 5.4.3
    Uninstalling ipykernel-5.4.3:
      Successfully uninstalled ipykernel-5.4.3


In [6]:
pip install simpletransformers

  Using cached https://files.pythonhosted.org/packages/7d/cc/e8908bbb2921732f6851ebbbe4b77b925aab62e644ab9402f21c84fa6107/ipykernel-5.4.3-py3-none-any.whl
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.4.3 which is incompatible.
  Found existing installation: ipykernel 4.10.0
    Uninstalling ipykernel-4.10.0:
      Successfully uninstalled ipykernel-4.10.0


In [37]:
import os 
from google.colab import drive
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/DATASET AML/PROJECT/dataset_giusti')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:

train = pd.read_csv("MULTILABEL_augmented_EDA_senza_stopword_FINAL.csv")
train["comment_text"] = train["comment_text"].astype(str)
train.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,well u say really hurt u block bitch tit ur pu...,1,1,1,0,1,0
1,dont know think cocksucker whole load shite ph...,1,1,1,0,1,0
2,robert palmer respect intend binknsternet kiss...,1,0,1,0,1,0
3,keep head man not let uppity bitch get man one...,1,0,1,0,1,0
4,naked mum sex indeed faggot suck tour dad cock...,1,1,1,0,1,1


In [42]:
labels_train = []
for i in range (len(train)):
    labels_train.append(train.iloc[i][1:7].values)

In [ ]:
labels_train

In [44]:
#train=train.rename(columns={"comment_text":"text", "bad":"labels"})
train["labels"]= pd.Series(labels_train)
train= train[["comment_text", "labels"]]
train=train.rename(columns={"comment_text":"text"})

In [45]:
train

,text,labels
0,well u say really hurt u block bitch tit ur pu...,"[1, 1, 1, 0, 1, 0]"
1,dont know think cocksucker whole load shite ph...,"[1, 1, 1, 0, 1, 0]"
2,robert palmer respect intend binknsternet kiss...,"[1, 0, 1, 0, 1, 0]"
3,keep head man not let uppity bitch get man one...,"[1, 0, 1, 0, 1, 0]"
4,naked mum sex indeed faggot suck tour dad cock...,"[1, 1, 1, 0, 1, 1]"
...,...,...
23176,not think gay may relationship guy big deal,"[1, 0, 0, 0, 0, 0]"
23177,buckeye love swallow sup state fuck lesbo,"[1, 0, 1, 0, 1, 1]"
23178,month later idiotic back forth continue dan as...,"[1, 0, 0, 0, 0, 0]"
23179,many chin hi faggot many chin exercise little ...,"[1, 0, 1, 0, 1, 0]"


In [31]:
def turn_int(x):
  return [int(s) for s in x[1:len(x)-1].split(' ')]

train["labels"]=train["labels"].apply(lambda x :turn_int(x))

In [13]:

os.chdir('/content/drive/My Drive/DATASET AML/PROJECT/dataset_giusti/TEST')
test = pd.read_csv("test_multilabel_senza_stopword.csv")


In [21]:
labels = []
for i in range (len(test)):
    labels.append(test.iloc[i][3:9].values)

In [26]:
#train=train.rename(columns={"comment_text":"text", "bad":"labels"})
test["labels"]= pd.Series(labels)
test= test[["comment_text", "labels"]]
test=test.rename(columns={"comment_text":"text"})

In [27]:
test.head()

,text,labels
0,arab commit genocide iraq protest europe may e...,"[1, 0, 0, 0, 0, 0]"
1,dj robinson gay hell suck dick much,"[1, 0, 1, 0, 1, 1]"
2,fuck antisemitic cunt,"[1, 0, 1, 0, 1, 0]"
3,dare vandalize page hm beagle not vandalize demon,"[1, 0, 0, 0, 0, 0]"
4,arrogant self serve immature idiot get right,"[1, 0, 1, 0, 1, 0]"


XLNET

In [28]:
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)

In [46]:
# Optional model configuration
model_args = MultiLabelClassificationArgs(num_train_epochs=5, overwrite_output_dir= True)

# Create a ClassificationModel
model= MultiLabelClassificationModel(
    "xlnet", "xlnet-base-cased", num_labels=6, args= model_args
)

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream tas

args = { 'model_type': 'bert', 'model_name': 'bert-base-cased', 'output_dir': 'outputs/', 'cache_dir': 'cache/', 'fp16': True, 'fp16_opt_level': 'O1', 'max_seq_length': 100, 'train_batch_size': 256, 'eval_batch_size': 256, 'gradient_accumulation_steps': 1, 'num_train_epochs': 1, 'weight_decay': 0, 'learning_rate': 4e-5, 'adam_epsilon': 1e-8, 'warmup_ratio': 0.06, 'warmup_steps': 0, 'max_grad_norm': 1.0, 'logging_steps': 50, 'evaluate_during_training': False, 'save_steps': 2000, 'eval_all_checkpoints': True, 'use_tensorboard': True, 'overwrite_output_dir': True, 'reprocess_input_data': False, }

In [47]:
# Train the model
model.train_model(train)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test)



/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


In [48]:
model_outputs

array([[9.85351562e-01, 1.04427338e-03, 6.38427734e-02, 9.82910156e-01,
        9.84375000e-01, 9.58496094e-01],
       [9.99511719e-01, 7.13378906e-01, 9.99023438e-01, 4.74691391e-04,
        9.95605469e-01, 9.30664062e-01],
       [9.97070312e-01, 9.95117188e-01, 9.98046875e-01, 2.09045410e-03,
        9.96093750e-01, 9.95117188e-01],
       ...,
       [8.26660156e-01, 8.65936279e-04, 5.15441895e-02, 8.10623169e-04,
        9.75097656e-01, 2.68363953e-03],
       [9.62890625e-01, 3.47375870e-04, 3.16162109e-02, 1.46508217e-04,
        1.17187500e-01, 8.89778137e-04],
       [9.95605469e-01, 3.16162109e-02, 9.99511719e-01, 3.10182571e-04,
        9.64355469e-01, 9.88281250e-01]])

In [49]:
result

{'LRAP': 0.9468071351024221, 'eval_loss': 0.42786730028851083}

In [50]:
wrong_predictions

['NA']